In [ ]:
!pip install --upgrade gensim
!pip install numpy
!pip install pandas==1.5.3
!pip install pyLDAvis

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import random
import gensim
from gensim.utils import simple_preprocess

from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel


import pyLDAvis

import pyLDAvis.gensim
pyLDAvis.enable_notebook()


import matplotlib.pyplot as plt




# Ejemplo con Reviews de Asistentes Virtuales

# Lectura de datos

In [ ]:
df = pd.read_csv('./reviews_sample.csv')



In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df = df[['utterance']]
df.dropna(inplace=True)

# Preprocesado

Lo hacemos con Gensim por "presentarlo", pero sabéis que opciones hay infinitas

In [ ]:
def text_preprocessing(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [ ]:
print('Original text:\n{}\n\n'.format(df['utterance'][0]))
print('Processed text:\n{}'.format(text_preprocessing(df['utterance'][0])))

In [ ]:
processed_texts = []
for text in df['utterance']:
    processed_texts.append(text_preprocessing(text))

In [ ]:
processed_texts[:10]

## Diccionario id - palabra

In [ ]:
dictionary = Dictionary(processed_texts)

In [ ]:
type(dictionary)

In [ ]:
list(dictionary.items())

In [ ]:
len(dictionary)

## Matriz documento-palabra

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in processed_texts]

In [ ]:
corpus

In [ ]:
corpus[0]

In [ ]:
corpus[5]

## Entrenamos el modelo (LDA)

In [ ]:
num_topics = 8

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    iterations=5,
    passes=10,
    alpha='auto'
)



In [ ]:
lda_model.show_topics()

In [ ]:
word_dict = {};
for i in range(num_topics):
    words = lda_model.show_topic(i, topn = 20)
    word_dict['Topic #' + '{:02d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

## Perplexity y Coherence

In [ ]:
# Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # Medida de qué tan bueno es el modelo. Cuanto más bajo, mejor

# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook ()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

In [ ]:
pyLDAvis.save_html(vis, './topics_vis_0.html')

## Número óptimo de topics

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):

        # Build LDA model
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=num_topics)

        # Create a list of LDA models
        model_list.append(model)

        # Compute the Coherence for each model
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

Grid sobre el número de topics (puede tardar un buen rato)

In [ ]:
start_ = 3
end_ = 14
step_ = 1

In [ ]:
model_list, coherence_values = compute_coherence_values(
    dictionary=dictionary,
    corpus=corpus,
    texts=processed_texts,
    start=start_,
    limit=end_,
    step=step_
)

In [ ]:
x = range(start_, end_, step_)
plt.plot(x, coherence_values)
plt.xlabel('Num Topics')
plt.ylabel('Coherence score')
plt.legend(('coherence_values'), loc='best')
plt.show()

In [ ]:
optimal_model_id = -1

optimal_model = model_list[optimal_model_id]

word_dict = {};
for i in range(0, optimal_model.num_topics , 1):
    words = optimal_model.show_topic(i, topn = 20)
    word_dict['Topic #' + '{:02d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

# Topic dominante por texto

In [ ]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    rows_to_append = []
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                rows_to_append.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]))
            else:
                break

    sent_topics_df = pd.concat(rows_to_append, axis=1).T if rows_to_append else pd.DataFrame()
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=processed_texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

# Documento más representativo por topic

In [ ]:
sent_topics_sorteddf_lda = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_lda = pd.concat([
        sent_topics_sorteddf_lda,
        grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)],
        axis=0)

# Reset Index
sent_topics_sorteddf_lda.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_lda.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_lda

# Distribución de topics en el corpus

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), num_topics)
df_topic_contribution = pd.DataFrame({'topic': topic_contribution.index, 'contribution': topic_contribution})
df_topic_contribution.reset_index(drop=True, inplace=True)

# Show
df_topic_contribution